In [98]:
import pandas as pd
import numpy as np

In [127]:
#Make toy dataframe to try mapping

#test_df ~= result of subset of columns in df made from genbank seqs
#each row is a sequence
country = ['United States', 'Chile', 'Spain', 'Japan', 'United States', 'Spain', 'Spain', 'Spain', 'Spain']
# country_code = ['USA', 'CHL', 'ESP', 'JPN', 'USA', 'ESP', 'ESP', 'ESP', 'ESP']
subtype= ['A','A','A','B','B','A', 'A', 'A', 'B']

test_df = pd.DataFrame({'country':country, 'subtype': subtype})

#use lat and long if country name parsing becomes confusing in plotly
#lat and long data from https://worldmap.harvard.edu/data/geonode:country_centroids_az8

lat_lon = pd.read_csv('country_centroids_az8.csv', \
                      usecols=['name','brk_a3','Longitude','Latitude']\
                     ).rename(columns={'name':'country', 'brk_a3':'country_code'})

#count number of rows(seqs) from each country that are each subtype
df_count = pd.DataFrame({'count' : test_df.groupby(['country', 'subtype']).size()}).reset_index()

#compile country-specific subtype count data with lat and long for plotting
df_countries = df_count.merge(lat_lon, how='left', left_on='country', right_on='country')

In [129]:
#Jitter points for countries that have multiple subtypes, so markers on map don't overlap
jitter_dict= {'A':1.0, 'B':-1.0}

test_group = df_countries.groupby('country').size()

df_countries['adj_lon'] = np.where(test_group[df_countries['country']]>1, 
                                   (df_countries['Longitude']+df_countries.subtype.map(lambda x: jitter_dict[x])
                                   ), df_countries['Longitude'])

df_countries['adj_lat'] = np.where(test_group[df_countries['country']]>1, 
                                   (df_countries['Latitude']+df_countries.subtype.map(lambda x: jitter_dict[x])
                                   ), df_countries['Latitude'])


In [131]:
import plotly.plotly as py
import plotly.graph_objs as go

scale_markers = 10
map_list = []
cmap= {'A':'royalblue','B':'salmon'}



for i in range(len(df_countries)):

    map_country = dict(
        type = 'scattergeo',
#         locationmode = 'country names',
#         locations = [df_countries.loc[i,'country']],
        lat = [df_countries.loc[i,'adj_lat']],
        lon = [df_countries.loc[i,'adj_lon']],
        marker = dict(
            size = df_countries.loc[i,'count']*scale_markers,
            color = cmap[df_countries.loc[i,'subtype']],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            opacity=0.5,
            sizemode = 'diameter'),
        hovertext = 'Subtype '+df_countries.loc[i,'subtype']+' : '+str(df_countries.loc[i,'count'])+' sequences',
        name = df_countries.loc[i,'country']+' '+df_countries.loc[i,'subtype'],
        hoverinfo = 'text+name'
    )
    map_list.append(map_country)

    
layout = dict(
        title = 'Global distribution of RSV',
        showlegend = False,
        geo = dict(
            scope='world',
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            countrywidth=1,
        ),
    )

fig = dict(data=map_list, layout=layout)
py.iplot(fig)

PlotlyRequestError: Account limit reached: Your account is limited to creating 25 charts. To continue, you can override or delete existing charts or you can upgrade your account at: https://plot.ly/products/cloud